## Daten Laden

In [1]:
file_path = "LITTLE_2025-03-12__06_10_28-normalized-cleaned-supplemented.csv"
print(file_path)

LITTLE_2025-03-12__06_10_28-normalized-cleaned-supplemented.csv


In [2]:
import pandas as pd

df = pd.read_csv(file_path)

# Drop command column
df = df.drop(columns=['command'])

# Display the first few rows to understand the structure
df.head()

,watt-median,L1-dcache-loads,L1-dcache-loads-root,L1-dcache-loads-square,L1-dcache-stores,L1-dcache-stores-root,L1-dcache-stores-square,L1-icache-loads,L1-icache-loads-root,L1-icache-loads-square,...,unc_m_dram_page_empty_rd-square,unc_m_vc0_requests_rd,unc_m_vc0_requests_rd-root,unc_m_vc0_requests_rd-square,unc_mc0_rdcas_count_freerun,unc_mc0_rdcas_count_freerun-root,unc_mc0_rdcas_count_freerun-square,unc_m_dram_page_hit_rd,unc_m_dram_page_hit_rd-root,unc_m_dram_page_hit_rd-square
0,2.145,4.402872e+04,209.830209,1.938528e+09,2.660100e+04,163.098143,7.076134e+08,5.415472e+04,232.711678,2.932734e+09,...,4.054235e+10,6.896262e+05,830.437364,4.755843e+11,3.447701e+05,587.171264,1.188664e+11,4.752240e+05,689.364945,2.258379e+11
1,10.550,1.619215e+09,40239.471539,2.621857e+18,6.951793e+08,26366.252209,4.832742e+17,1.523664e+09,39034.133275,2.321551e+18,...,2.773084e+12,3.883205e+06,1970.584935,1.507928e+13,2.017283e+06,1420.310741,4.069429e+12,2.334088e+06,1527.772312,5.447968e+12
2,9.530,4.332416e+08,20814.455361,1.876982e+17,1.508824e+08,12283.419289,2.276550e+16,1.511742e+08,12295.290855,2.285363e+16,...,6.652009e+14,2.580212e+07,5079.578715,6.657494e+14,1.333257e+07,3651.378637,1.777573e+14,4.110717e+05,641.148708,1.689799e+11
3,9.375,6.226269e+08,24952.493274,3.876643e+17,2.820873e+08,16795.454838,7.957325e+16,8.146840e+08,28542.670656,6.637101e+17,...,1.852295e+09,1.125976e+05,335.555631,1.267822e+10,5.711226e+04,238.981718,3.261810e+09,6.946080e+04,263.554173,4.824803e+09
4,9.645,1.040844e+09,32262.118982,1.083357e+18,1.039464e+09,32240.719618,1.080485e+18,2.247217e+09,47404.821850,5.049985e+18,...,9.722586e+08,8.143194e+04,285.362815,6.631160e+09,4.145827e+04,203.613046,1.718788e+09,5.002804e+04,223.669485,2.502805e+09


## Korrelationsmatrix

In [3]:
corr = df.corr()
corr.style.background_gradient(cmap='RdYlGn')

## 1. Versuch (Regression)

In [4]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Zielvariable: Annahme, dass die erste Spalte die zu schätzende Zahl enthält
y = df.iloc[:, 0]

# Feature, das immer genutzt werden soll: 'instructions'
X_instructions = df[['instructions']]

# Potenzielle zusätzliche Features: Alle Spalten außer der Zielvariable und 'instructions'
potential_features = df.drop(columns=[df.columns[0], 'instructions'])

# Automatische Auswahl der 4 besten Features basierend auf f_regression
selector = SelectKBest(score_func=f_regression, k=4)
X_selected = selector.fit_transform(potential_features, y)

# Ausgabe der Namen der ausgewählten Features
selected_feature_names = potential_features.columns[selector.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Falls gewünscht: Umwandeln in DataFrame, um die Spaltennamen beizubehalten
X_selected_df = pd.DataFrame(X_selected, columns=selected_feature_names, index=df.index)

# Kombination der 'instructions'-Spalte mit den ausgewählten Features
X_combined = pd.concat([X_instructions, X_selected_df], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Training eines Regressionsmodells (z. B. Lineare Regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))


Ausgewählte Features: ['cpu-cycles-square', 'ref-cycles-square', 'msr/aperf/-square', 'msr/mperf/-square']
Mean Squared Error: 1.6195778754956516


## 2. Versuch (Regression)

### Modell trainieren

In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Zielvariable: erste Spalte (angenommen, es handelt sich um numerische Werte)
y = df.iloc[:, 0]

# Fixes Feature: 'instructions'
X_instructions = df[['instructions']]

# Kandidaten für die automatische Feature-Auswahl:
# Entfernen der Zielvariable und der Spalte 'instructions'
X_candidates = df.drop(columns=[df.columns[0], 'instructions'])

# Basisregressor für die Feature-Selektion (hier: Lineare Regression)
model = LinearRegression()

# Sequential Feature Selector (forward selection) wählt 4 zusätzliche Features aus
sfs = SequentialFeatureSelector(
    model,
    n_features_to_select=4,
    direction='forward',
    scoring='neg_mean_squared_error',
    cv=5  # 5-fache Kreuzvalidierung
)

# Durchführung der Feature-Selektion auf den Kandidatenfeatures
sfs.fit(X_candidates, y)

# Ermitteln der Namen der ausgewählten Features
selected_feature_names = X_candidates.columns[sfs.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Zusammenführen des fixen Features 'instructions' mit den ausgewählten Features
X_selected = pd.concat([X_instructions, X_candidates[selected_feature_names]], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Modelltraining auf den kombinierten Features
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

# Ausgabe der Regressionsparameter
print("\nRegression Parameter:")
print("Achsenabschnitt (Intercept):", model.intercept_)

# Ausgabe der Koeffizienten mit den zugehörigen Feature-Namen
print("Koeffizienten:")
for feature, coef in zip(X_selected.columns, model.coef_):
    print(f"{feature}: {coef}")

Ausgewählte Features: ['L1-dcache-loads', 'topdown-be-bound-root', 'topdown-retiring-root', 'unc_m_clockticks-square']
Mean Squared Error: 30.94111245984888

Regression Parameter:
Achsenabschnitt (Intercept): 27.97667106209125
Koeffizienten:
instructions: -8.149589634164032e-12
L1-dcache-loads: 3.993038774926663e-10
topdown-be-bound-root: -4.655073953991698e-06
topdown-retiring-root: 4.095133218960428e-06
unc_m_clockticks-square: -3.287145824941493e-18


### Modell testen

In [6]:
def do_regression(instructions, L1_dcache_loads, L1_icache_loads, topdown_retiring, uncore_clock_clockticks):
  return 5.671202142645853 + instructions * -4.037434477408138e-11 + L1_dcache_loads * 3.7727498541646285e-10 + topdown_retiring * 9.090283761268329e-11 + uncore_clock_clockticks * 1.4513036090171807e-09

# sleep 10 (expected 2.14)
print("sleep 10 (expected 2.14)")
print(do_regression(165556.153198144, 44181.0547038958, 54556.2364921857, 208105.052597547, 38000929.1425307))

print("")

# sum_up_benchmark (expected 10.46)
print("sum_up_benchmark (expected 10.46)")
print(do_regression(7658080659.63333, 1628055678.50196, 1528910300.65665, 7961570136.8392, 2696560318.9496))

print("")

# stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)
print("stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)")
print(do_regression(9439119304.88239, 1242411937.11209, 1632001751.87898, 9934301707.13731, 2690003038.87471))

sleep 10 (expected 2.14)
5.726381929781779

sum_up_benchmark (expected 10.46)
10.613493876265267

stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)
10.565902164387403


## 3. Versuch (Schwierig auf Colab...)

### Modell trainieren

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class CustomSelector(BaseEstimator, TransformerMixin):
    """
    Transformer, der die Spalte 'instructions' immer mit einbezieht
    und zusätzlich eine Liste von weiteren Features auswählt, die als Parameter übergeben wird.
    """
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Es werden immer 'instructions' plus die angegebenen features zurückgegeben
        return X[['instructions'] + list(self.features)]

def optimal_feature_selection_regression_cv(df, test_size=0.2, random_state=42):
    """
    Verwendet GridSearchCV, um die optimale Feature-Kombination zu finden.
    'instructions' wird stets genutzt und zusätzlich werden 4 weitere Features
    aus den Kandidaten (alle außer der Zielvariable und 'instructions') ausgewählt.

    Rückgabe:
      dict: Mit den besten zusätzlichen Features, dem CV-Score,
            Test-MSE, Achsenabschnitt und Koeffizienten.
    """
    # Zielvariable: erste Spalte
    y = df.iloc[:, 0]
    # Prädiktoren: alle Spalten außer der Zielvariable
    X = df.drop(columns=[df.columns[0]])

    # Kandidaten für zusätzliche Features: alle außer 'instructions'
    candidates = list(X.drop(columns=['instructions']).columns)

    # Pipeline: CustomSelector + LinearRegression
    pipeline = Pipeline([
        ('selector', CustomSelector(features=[])),
        ('regressor', LinearRegression())
    ])

    # Erzeuge den Parametergrid: alle Kombinationen von 4 Features aus den Kandidaten
    param_grid = {
        'selector__features': [list(comb) for comb in combinations(candidates, 4)]
    }

    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X, y)

    best_features = grid.best_params_['selector__features']
    best_score = grid.best_score_

    print("Beste zusätzliche Features:", best_features)
    print("Bester CV Score (neg_mean_squared_error):", best_score)

    # Aufteilen in Trainings- und Testdaten
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Bestes Modell trainieren
    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Finales Test-MSE:", mse)

    # Regressionsparameter ausgeben
    regressor = best_model.named_steps['regressor']
    print("\nRegression Parameter:")
    print("Achsenabschnitt (Intercept):", regressor.intercept_)

    coef_dict = {}
    # Die verwendeten Spalten: 'instructions' plus die best_features
    features_used = ['instructions'] + best_features
    print("Koeffizienten:")
    for feature, coef in zip(features_used, regressor.coef_):
        print(f"{feature}: {coef}")
        coef_dict[feature] = coef

    return {
        "best_features": best_features,
        "cv_score": best_score,
        "test_mse": mse,
        "intercept": regressor.intercept_,
        "coefficients": coef_dict
    }

# Beispiel zur Nutzung:
# df = pd.read_csv('data.csv')
# result = optimal_feature_selection_regression_cv(df)
# print(result)

optimal_feature_selection_regression_cv(df)

### Modell testen

In [ ]:
def do_regression(instructions, L1_dcache_stores, dTLB_loads, dTLB_stores, uncore_clock_clockticks):
  return 20.835376283133087 + instructions * 7.719055169945736e-11 + L1_dcache_stores * 6.750946704125803e-09 + dTLB_loads * 5.752769284893428e-10 + dTLB_stores * -6.910438337198133e-09 + uncore_clock_clockticks * -4.169191670737849e-09

# sleep 10 (expected 2.14)
print("sleep 10 (expected 2.14)")
print(do_regression(165556.153198144, 26701.7192578545, 44249.7939094154, 26902.9815858347, 38000929.1425307))

print("")

# sum_up_benchmark (expected 10.46)
print("sum_up_benchmark (expected 10.46)")
print(do_regression(7658080659.63333, 698890004.553369, 1626382945.67966, 698381494.410173, 2696560318.9496))

print("")

# stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)
print("stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)")
print(do_regression(9439119304.88239, 561931144.341175, 1237133407.39931, 559806745.508526, 2690003038.87471))